In [62]:
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from nltk import word_tokenize

# from gensim.models.ldamulticore import LdaMulticore  # for big dataset, use multiple core machine
import gensim
from gensim.test.utils import common_texts
from gensim import corpora, models
from gensim.corpora.dictionary import Dictionary
from gensim.models import Phrases
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
from nltk.stem import WordNetLemmatizer

from nltk.corpus import stopwords
stops = stopwords.words("english")
import re

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
%matplotlib inline
# import pyLDAvis
# import pyLDAvis.gensim_models as gensimvis

!pip install ipywidgets

In [9]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [4]:
from NLP_Natural_Disasters.data import get_data, clean_data

### exemple Semi-suppervised LDA sur Kaggle

https://www.kaggle.com/code/vanguarde/semi-supervised-lda

In [7]:
def text_cleaning(texts):
    texts_cleaning = []
    for txt in tqdm(texts):
        url = re.compile(r'https?://\S+|www\.\S+')
        html = re.compile(r'<.*?>')
        emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
        txt = emoji_pattern.sub(r'', txt)
        txt = html.sub(r'',txt)
        txt = url.sub(r'',txt)
        txt = re.sub('[^A-Za-z\s]', '', txt)
        
        texts_cleaning.append(txt.lower())
    return texts_cleaning

text = text_cleaning(train_df.text.tolist())

NameError: name 'train_df' is not defined

In [ ]:
from nltk.stem import PorterStemmer
text = [t.split() for t in text]
stemmed_text = []
ps = PorterStemmer()
for sentence in tqdm(text):
    sent = []
    for word in sentence:
        sent.append(ps.stem(word))
    stemmed_text.append(sent)

In [ ]:
print(*stemmed_text[5][:20])
print(*text[5][:20])

In [ ]:
dictionary = gensim.corpora.Dictionary(text_split)

IF doesn't work : dictionary.doc2bow  --> tqdm
tried solution :
conda install -c conda-forge ipywidgets
jupyter nbextension enable --py widgetsnbextension
SOLUTION : remove 'in tqdm' add a splitted list of sentences

In [ ]:
stopword_ids = map(dictionary.token2id.get, stops)
dictionary.filter_tokens(bad_ids=stopword_ids)
dictionary.filter_extremes(no_below=20, no_above=0.7, keep_n=None)
dictionary.compactify() # remove gaps in id sequence
bow = [dictionary.doc2bow(line) for line in text_split]

In [ ]:
disasters = ['disaster', 'bloodbath', 'collapse', 'crash', 'meltdown', 'doomsday', 'convulsion', 'accident', 'casualty', 'fatality', 
            'blast', 'catastrophe', 'traffic','hybrid', 'engine', 'license', 
            'tsunami', 'volcano','tornado','avalanche','earthquake','blizzard','drought','bushfire','tremor','magma','twister',
            'windstorm','cyclone','flood','fire','hailstorm','lava','lightning','hail','hurricane','seismic','erosion','whirlpool','whirlwind',
            'cloud','thunderstorm','barometer','gale','blackout','gust','force','volt','snowstorm','rainstorm','storm','nimbus','violent storm',
            'sandstorm','fatal','cumulonimbus','death','lost','destruction','money','tension','cataclysm','damage','uproot','underground',
            'destroy','arsonist','wind scale','arson','rescue','permafrost','fault','shelter', 'bomb', 'suicide', 'tragedy', 'weapon']

disasters = [ps.stem(word) for word in disasters]

# add more topics later on

In [ ]:
seed_topics = {}
for word in disasters:
    seed_topics[word] = 0

In [ ]:
def create_eta(priors, etadict, ntopics):
    eta = np.full(shape=(ntopics, len(etadict)), fill_value=1) # create a (ntopics, nterms) matrix and fill with 1
    for word, topic in priors.items(): # for each word in the list of priors
        keyindex = [index for index,term in etadict.items() if term==word] # look up the word in the dictionary
        if (len(keyindex)>0): # if it's in the dictionary
            eta[topic,keyindex[0]] = 1e7  # put a large number in there
    eta = np.divide(eta, eta.sum(axis=0)) # normalize so that the probabilities sum to 1 over all topics
    return eta

In [ ]:
eta = create_eta(seed_topics, dictionary, 2)

In [ ]:
eta

For now, Number of topics = 2: disasters , common topic

In [ ]:
# random_state=42
lda_model_core = LdaMulticore(corpus=bow, #bag of words
                         id2word=dictionary, #our common dict, need for print words in topics, not numbers from bow
                         num_topics=2,
                         eta=eta, #our eta matrix
                         chunksize=2000,
                         passes=10,
                         random_state=42,
                         alpha='symmetric', #param of LDA distribution. If you dont know use symmetric
                         per_word_topics=True)

In [ ]:
for num, params in lda_model_core.print_topics():
    print(f'{num}: {params}\n')

### LDA Topic Modelisation avec Gensim :  Creer des topics (pertinent!) , analyse mot -> classifier

In [5]:
## EDA cleaning process

data = get_data()
data_clean = clean_data(data)

data_clean.head()

,id,text,target
0,1,deed reason earthquake may allah forgive u,1
1,4,forest fire near la canada,1
2,5,resident asked shelter place officer evacuatio...,1
3,6,people receive wildfire evacuation order calif...,1
4,7,got sent photo alaska smoke wildfire school,1


In [6]:
X_train = data_clean[data_clean['target']==1]
X_train = X_train['text']

# transform to a list, split to get list of each sentence splitted for gensim.corpora.Dict
list_tweet = X_train.tolist()
text_split = [x.split() for x in list_tweet]

dictionary = gensim.corpora.Dictionary(text_split)

In [7]:
# Transformation to create a Bag of Words .doc2bow for LdaModel later
# stopword_ids = map(dictionary.token2id.get, stops)
# dictionary.filter_tokens(bad_ids=stopword_ids)
# dictionary.filter_extremes(no_below=20, no_above=0.7, keep_n=None)
# dictionary.compactify() # remove gaps in id sequence
bow = [dictionary.doc2bow(line) for line in text_split]

In [83]:
from joblib import dump, load
dump(dictionary, 'bow_dict.joblib') 

['bow_dict.joblib']

### Analysis & Research - crash test !

#### Topic analysis : Raw data

In [ ]:
target_1_raw = data[data['target']==1]
target_1_raw = target_1_raw['text'] 

In [ ]:
pd.DataFrame(target_1_raw)

#### Gensim LDA model syntax tuto

https://radimrehurek.com/gensim/models/ldamodel.html#gensim.models.ldamodel.LdaModel

In [ ]:
"""Train an LDA model using a Gensim corpus"""

# Create a corpus from a list of texts
common_dictionary = Dictionary(common_texts)
common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]

# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=10)


    """Save a model to disk, or reload a pre-trained model"""
from gensim.test.utils import datapath

# Save model to disk.
temp_file = datapath("model")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)


    """Query, the model using new, unseen documents"""
# Create a new corpus, made of previously unseen documents.
other_texts = [
    ['computer', 'time', 'graph'],
    ['survey', 'response', 'eps'],
    ['human', 'system', 'computer']
]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[0]
vector = lda[unseen_doc]  # get topic probability distribution for a document


    """Update the model by incrementally training on the new corpus"""
lda.update(other_corpus)
vector = lda[unseen_doc]


    """A lot of parameters can be tuned to optimize training for your specific case"""
lda = LdaModel(common_corpus, num_topics=50, alpha='auto', eval_every=5)  # learn asymmetric alpha from data

### Create my own topics

> countvectoriser most_commun : tableau avec le compte totalde chaque mot et son occurence 

In [8]:
# list to create my 3 topic, 3rd is random other subjects

# this list means need forces intervention or involved
forces_intervention = ['accident', 'officers', 'evacuation', 'shelter', 'wildfire', 'wildfires', 'smoke', 'fire', 'emergency', 'die', 'died' 'ablaze', 'burn', 'burned',
'crash', 'accident', 'traffic', 'damage', 'deadly', 'kill', 'killed', 'explosion', 'death', 'ambulance', 'drunk', 'crime', 'attack', 'attacked', 'victim', 'stabbed', 'stabbing', 'arson',
'arrested', 'arrestation', 'burning', 'arsonist', 'collided', 'collision', 'police', 'suicide', 'bombing', 'injured', 'terrorist', 'avalanche', 'army', 'derailed', 'stab', 'blew', 'collapse']

weather_phenomen = ['eruption', 'earthquake', 'catastrophic effect','heavy', 'rain', 'flood', 'tornado', 'heat', 'tsunami', 'volcano','tornado', 'magma', 'windstorm','cyclone','hailstorm','lava','hail','hurricane','seismic','erosion',
'whirlwind', 'cloud','thunderstorm','barometer','gale','blackout','gust','snowstorm','rainstorm','storm','nimbus','violent storm', 'sandstorm','cumulonimbus']

# ps = PorterStemmer()
lemmatizer  = WordNetLemmatizer()
forces_intervention = [lemmatizer.lemmatize(word) for word in forces_intervention]
weather_phenomen = [lemmatizer.lemmatize(word) for word in weather_phenomen]

In [ ]:
## just to see, take a word at its root : killed -> kill
# forces_intervention

**Placé dans target1**
'star wars',  'Indeed!! I am fully aware of that battle! I support you in that fight!'

menace : 'you can stab me in the back but I promise you'll be the one bleeding'

news : 'transport bioterror germs in wake of anthrax'

suspect = ['afraid', 'hate', 'bomb']

In [9]:
# initialize value 0 to each word to go through
# then we'll use create_eta() function to add a big value if word in lists above

seed_topics_forces = {}

for word in forces_intervention:
    seed_topics_forces[word] = 0
    
seed_topics_weather = {}

for word in weather_phenomen:
    seed_topics_weather[word] = 1

In [ ]:
# just to see
# seed_topics_forces

Custom **eta** parameter of gensim.models.ldamodel.LdaModel (not sklrean ldamodel)

In [10]:
def create_eta(dict_priors, etadict, ntopics):
    """ create 'eta' parameter in LdaModel,
    dict_priors  is a list of dicts from see_topics """
    eta = np.full(shape=(ntopics, len(etadict)), fill_value=1) # create a (ntopics, nterms) matrix and fill with 1
    
    for dict_prior in dict_priors:
        
        for word, topic in dict_prior.items(): # for each word in the list of priors
            
            keyindex = [index for index,term in etadict.items() if term==word] # look up the word in the dictionary
            if (len(keyindex)>0): # if it's in the dictionary
                eta[topic,keyindex[0]] = 1e7  # put a large number in there
                
    eta = np.divide(eta, eta.sum(axis=0)) # normalize so that the probabilities sum to 1 over all topics
    return eta

In [11]:
# create my own eta, affect a big value to a word in my seed_topics_force, seed_topic_weather
# IF is in my lists created above (force_intervention, weather_phenomen')
eta = create_eta([seed_topics_forces, seed_topics_weather], dictionary, 3)

Define LdModel() : gensim.models.ldamodel.LdaModel

### LDA model custom

class **gensim.models.ldamodel.LdaModel**(corpus=None, num_topics=100, id2word=None, distributed=False, chunksize=2000, passes=1, update_every=1, alpha='symmetric', eta=None, decay=0.5, offset=1.0, eval_every=10, iterations=50, gamma_threshold=0.001, minimum_probability=0.01, random_state=None, ns_conf=None, minimum_phi_value=0.01, per_word_topics=False, callbacks=None, dtype=<class 'numpy.float32'>)

**random_state** ({np.random.RandomState, int}, optional) – Either a randomState object or a seed to generate one. Useful for reproducibility.
random_state - this serves as a seed (in case you wanted to repeat exactly the training process)
**chunksize** - number of documents to consider at once (affects the memory consumption)

**update_every** - update the model every update_every chunksize chunks (essentially, this is for memory consumption optimization)

**passes** - how many times the algorithm is supposed to pass over the whole corpus

chunksize - number of documents to consider at once (affects the memory consumption)

In [54]:
num_topics=3
chunksize=2000
passes = 40
random_state = 100


lda_model = gensim.models.ldamodel.LdaModel(corpus=bow, #bag of words
                         id2word=dictionary, #our common dict, need for print words in topics, not numbers from bow
                         num_topics=num_topics,
                         eta=eta, #our eta matrix
                         chunksize=chunksize,
                        update_every = 1,
                         passes=passes,
                         random_state=random_state,
                         alpha ='asymmetric', #param of LDA distribution. If you dont know use symmetric
                        minimum_probability = 0.0,
                         per_word_topics=True,
                        dtype =np.float32,
                        iterations = 400,
                        eval_every = None)

# alpha ='symmetric' return SAME percentage, auto/asymmetric : return diff results, 
# random_state : serves as a seed (in case you wanted to repeat exactly the training process)
# n_jobs = -1 ?
# random_state=100,

In [80]:
from gensim.test.utils import datapath

# Save model to disk.
temp_file = datapath("/home/dianavo/code/AxelCatelan/NLP_Natural_Disasters/model_lda")

lda_model.save(temp_file)

In [81]:
model_test = LdaModel.load(temp_file)

(!) alpha ='symmetric' return SAME percentage --> set to auto/asymmetric : return diff % dispersion

the corpus contains the word id and its frequency in every document. We can create a BoW corpus from a simple list of documents and from text files. What we need to do is, to pass the tokenised list of words to the object named Dictionary. doc2bow().

In [31]:
# print 10 words and its percentage in each topics (0-2)

# print_topics(num_topics=20, num_words=10)
for num, params in lda_model.print_topics(num_words=25):
    print(f'{num}: {params}\n')

0: 0.010*"fire" + 0.008*"news" + 0.007*"california" + 0.007*"suicide" + 0.006*"family" + 0.006*"via" + 0.006*"crash" + 0.005*"pm" + 0.005*"home" + 0.005*"killed" + 0.005*"wildfire" + 0.005*"police" + 0.004*"video" + 0.004*"two" + 0.004*"legionnaire" + 0.004*"accident" + 0.004*"mh" + 0.004*"bomber" + 0.004*"people" + 0.003*"emergency" + 0.003*"kill" + 0.003*"get" + 0.003*"attack" + 0.003*"flood" + 0.003*"warning"

1: 0.010*"fire" + 0.009*"hiroshima" + 0.007*"like" + 0.007*"u" + 0.007*"amp" + 0.007*"year" + 0.006*"train" + 0.006*"storm" + 0.006*"building" + 0.006*"atomic" + 0.005*"japan" + 0.005*"still" + 0.005*"im" + 0.005*"today" + 0.005*"life" + 0.005*"people" + 0.004*"bomb" + 0.004*"look" + 0.004*"bombing" + 0.004*"first" + 0.004*"mass" + 0.004*"day" + 0.004*"burning" + 0.004*"derailment" + 0.004*"war"

2: 0.020*"disaster" + 0.011*"obama" + 0.009*"oil" + 0.008*"may" + 0.008*"spill" + 0.007*"u" + 0.007*"charged" + 0.007*"boy" + 0.006*"nuclear" + 0.005*"sign" + 0.005*"declares" + 0.005

In [ ]:
## afficher plus de mots pour chaque topic
## pq que des 1, verif autres codes, frequences? param, get_term_topics()   pq : (78, 1)

**First try : FAILED ! code deleted :(**

### Visualizing the entity recongnizer¶

In [ ]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher

# 1. Loading the language library
nlp = spacy.load('en_core_web_sm')
text = u'''
Elon Musk, the billionaire CEO of Tesla and SpaceX, is now the richest person in the world, surpassing former titleholder and Amazon chief Jeff Bezos with a net worth of $189.7 billion, according to Forbes’s real-time billionaire net-worth estimates on Jan. 8, 2021 at 1pm.
2 - Since March, Musk’s wealth has grown almost seven-fold, up a staggering $163.1 billion.
'''
# 2. Building a Pipline Object
doc = nlp(text)


In [ ]:
displacy.render(doc, style='dep', jupyter=True, options={'distance':90})

In [ ]:
displacy.render(doc, style='ent', jupyter=True)

Autre plot fancy :

In [ ]:

from time import time
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

n_samples = 2000
n_features = 1000
n_components = 3
n_top_words = 20


def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(1, 3, figsize=(30, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[: -n_top_words - 1 : -1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f"Topic {topic_idx +1}", fontdict={"fontsize": 30})
        ax.invert_yaxis()
        ax.tick_params(axis="both", which="major", labelsize=20)
        for i in "top right left".split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

data_samples = list_tweet

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(
    max_df=0.95, min_df=2, max_features=n_features, stop_words="english"
)
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()


print(
    "\n" * 2,
    "Fitting LDA models with tf features, n_samples=%d and n_features=%d..."
    % (n_samples, n_features),
)
lda = LatentDirichletAllocation(
    n_components=n_components,
    max_iter=5,
    learning_method="online",
    learning_offset=50.0,
    random_state=0,
)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

tf_feature_names = tf_vectorizer.get_feature_names_out()
plot_top_words(lda, tf_feature_names, n_top_words, "Topics in LDA model")

### Get_document_topics :    Exemple with an input test

Objectif : comprendre "parfaitement" gensim.ldamodel (not sklearn Lda)

1) faire marcher la pred >> voir le topic utilise les bon mots >> sinon regler **alpha** pour la distribution des topics 

Objectif 2 (short on time) Mieux classer : 

2) countvectoriser >> determiner l'occurence des mot en pair >> les mettre dans mes list de topics

> **get_document_topics** : Get the most relevant topics to the given word.

In [33]:
# also transform my test phrase
dictionary.doc2bow(['airplane','crash'])

[(41, 1), (77, 1)]

In [14]:
# def countvecto(allsentences):
#     """Vectorise sentences, return an array"""
#     vectorizer = CountVectorizer()
#     # tokenize and build vocab    
#     X_test = vectorizer.transform(allsentences)
#     return X_test
#     # old ver : return X_test.toarray()

def split_sentence(sentence):
    """split ONE sentence in individual words"""
    sentence_spl = [x.split() for x in sentence]
    return sentence_spl

**Re-run an lda model** to fit only on the sentence test, not the entire data train text

In [15]:
# def predict_topic(splitted_sentences):
#     """Predict the corresponding topic with a list of sentences cleaned (stopwords) and splitted"""
#     dictionary_pred = corpora.Dictionary(splitted_sentences)
#     # dictionary.filter_extremes(no_below=20, no_above=0.7, keep_n=None)
#     bow_pred = dictionary_pred.doc2bow(splitted_sentences)

#     # old ver : predict_topic = lda_model.get_document_topics(bow_pred)
#     predict_topic = lda_model_exemple.get_document_topics(bow_pred)
    
#     return predict_topic

> steps : lower, tokenize, doc2bow, get_document_topics

In [73]:
## INPUT an exemple as a list of ONE + CLEAN sentence ['man put fire school']
exemple = ['horrible accident man died in wings of airplane eruption']
#exemple = ['storm heavy rain']

In [74]:
my_exemple_splitted = split_sentence(exemple)

In [75]:
my_exemple_splitted

[['horrible',
  'accident',
  'man',
  'died',
  'in',
  'wings',
  'of',
  'airplane',
  'eruption']]

In [76]:
# also transform my test phrase
exemple_bow = [dictionary.doc2bow(line) for line in my_exemple_splitted]
exemple_bow

[[(62, 1), (118, 1), (148, 1), (184, 1), (246, 1), (1563, 1)]]

In [87]:
# use the function countvecto() to vectorize the sentence, fit in lda_model (Not fit_transform)
# exemple_vecto = countvecto(my_exemple_splitted[0])  ### OR exemple ?


In [77]:
# predict the probability in each topic using predict_topic()
predict_topic = lda_model.get_document_topics(exemple_bow)
pourcentage = [element for element in predict_topic]
pourcentage

[[(0, 0.9241476), (1, 0.04419785), (2, 0.03165454)]]

In [82]:
# test model saved
predict_topic = model_test.get_document_topics(exemple_bow)
pourcentage = [element for element in predict_topic]
pourcentage

[[(0, 0.9241405), (1, 0.04420494), (2, 0.031654544)]]

In [ ]:
test_predict = lda_model[exemple_bow]
a = [element for element in test_predict]
a

In [97]:
print(f"topic 1 Forces intervention, need or involved : {pourcentage[0]} %")
print(f"topic 2 Weather phenomen : {pourcentage[1]} %")
print(f"topic 3 Rlandom topics : {pourcentage[2]} %")

topic 1 Forces intervention, need or involved : [(0, 0.7957197), (1, 0.024716612), (2, 0.17956367)] %


IndexError: list index out of range

In [ ]:
# # To plot at Jupyter notebook
# pyLDAvis.enable_notebook()
# plot = pyLDAvis.prepare(lda_model, bow, dictionary, vocab=)
# # Save pyLDA plot as html file
# pyLDAvis.save_html(plot, 'LDA_NYT.html')
# plot

> Coherence value : Choosing the Best Number of Topics

In [ ]:
# def compute_coherence_values(dictionary, corpus, texts, 
#                              cohere, limit, start=2, step=2):
#     iterations = 100
#     temp = dictionary[0]
#     id2word = dictionary.id2token

#     coherence_values = []

#     for num_topics in range(start, limit, step):
#         model = LdaModel(corpus=corpus, 
#                          id2word=dictionary, 
#                          num_topics=num_topics,
#                          chunksize=chunksize,
#                          alpha='auto',
#                          eta='auto',
#                          iterations=iterations,
#                          passes=passes,
#                          eval_every=1,
#                          random_state=42,)
#         coherencemodel = CoherenceModel(model=model, 
#                                         texts=texts, 
#                                         dictionary=dictionary, 
#                                         coherence=cohere)
#         coherence_values.append(coherencemodel.get_coherence())

#     return coherence_values

In [ ]:
##### 30 min to run, in 8G RAM ####

# limit=50
# start=2
# step=2
# corpus = bow
# texts = text_split

# coherence_values = compute_coherence_values(dictionary=dictionary, 
#                                             corpus=corpus, 
#                                             texts=texts, 
#                                             cohere='c_v', 
#                                             start=start, 
#                                             limit=limit, 
#                                             step=step)

# plt.figure(figsize=(8,5))

# # Create a custom x-axis
# x = range(start, limit, step)

# # Build the line plot
# ax = sns.lineplot(x=x, y=coherence_values, color='#238C8C')

# # Set titles and labels
# plt.title("Best Number of Topics for LDA Model")
# plt.xlabel("Num Topics")
# plt.ylabel("Coherence score")
# plt.xlim(start, limit)
# plt.xticks(range(2, limit, step))

# # Add a vertical line to show the optimum number of topics
# plt.axvline(x[np.argmax(coherence_values)], 
#             color='#F26457', linestyle='--')

# # Draw a custom legend
# legend_elements = [Line2D([0], [0], color='#238C8C', 
#                           ls='-', label='Coherence Value (c_v)'),
#                    Line2D([0], [1], color='#F26457', 
#                           ls='--', label='Optimal Number of Topics')]

# ax.legend(handles=legend_elements, loc='upper right')

Display topics visually